In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, TimeDistributed, Flatten, BatchNormalization
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

# 데이터셋 불러오기
file_path = 'C:\\Users\\co279\\mp_statcast.csv'
data = pd.read_csv(file_path)

# Drop unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Fill missing values with 0
data = data.fillna(0)

if 'pitch_hand' in data:
    data = pd.get_dummies(data, columns=['pitch_hand'], drop_first=True)

# 2020, 2021, 2022년에 모두 존재하는 player_id 추출
data_2016 = data[data['year'] == 2016]
data_2017 = data[data['year'] == 2017]
data_2018 = data[data['year'] == 2018]
data_2019 = data[data['year'] == 2019]

player_ids_2016 = set(data_2016['player_id'].unique())
player_ids_2017 = set(data_2017['player_id'].unique())
player_ids_2018 = set(data_2018['player_id'].unique())
player_ids_2019 = set(data_2019['player_id'].unique())

common_player_ids = player_ids_2016 & player_ids_2017 & player_ids_2018 & player_ids_2019

# 공통 player_id에 해당하는 데이터 추출
common_data = data[data['player_id'].isin(common_player_ids)]

# 2020, 2021, 2022년에 해당하는 데이터만 추출
final = common_data[common_data['year'].isin([2016, 2017, 2018])]
final = final.sort_values(by=['player_id', 'year'])

# 필요한 컬럼 선택 (year 제외)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# 독립변수와 종속변수 분리
X = final[features].values
y = final[target].values

# 데이터 스케일링
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# 시계열 데이터 형태로 변환
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 3  # 시퀀스 길이 설정
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# 학습 데이터와 전체 데이터를 동일하게 설정
X_train, y_train = X_seq, y_seq

In [2]:
# LSTM_A

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_A
    model_A = Sequential()
    model_A.add(LSTM(128, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_A.add(LSTM(128, return_sequences=True))
    model_A.add(Dropout(rate=0.5))
    model_A.add(Flatten())
    model_A.add(Dense(1024, activation="relu"))
    model_A.add(Dropout(rate=0.5))
    model_A.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_A.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_A = model_A.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_19 = common_data[common_data['year'] == 2019]

    # 2023년 데이터 스케일링
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_A = model_A.predict(X_2019_seq)

    # 스케일 복원
    y_pred_A = scaler_y.inverse_transform(y_pred_scaled_A)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_19[target].values

    # RMSE 계산
    rmse_A = np.sqrt(mean_squared_error(y_test_actual, y_pred_A))
    rmse_list.append(rmse_A)

    # MAE 계산
    mae_A = mean_absolute_error(y_test_actual, y_pred_A)
    mae_list.append(mae_A)
    
    # MAPE 계산
    mape_A = mean_absolute_percentage_error(y_test_actual, y_pred_A)
    mape_list.append(mape_A)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_A}, MAE: {mae_A}, MAPE: {mape_A}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
7/7 [==============================] - 9s 302ms/step - loss: 0.0789 - accuracy: 0.0025 - val_loss: 0.0464 - val_accuracy: 0.0000e+00
Epoch 2/500
7/7 [==============================] - 0s 26ms/step - loss: 0.0361 - accuracy: 0.0025 - val_loss: 0.0290 - val_accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 25ms/step - loss: 0.0266 - accuracy: 0.0025 - val_loss: 0.0374 - val_accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 30ms/step - loss: 0.0220 - accuracy: 0.0025 - val_loss: 0.0284 - val_accuracy: 0.0000e+00
Epoch 5/500
7/7 [==============================] - 0s 23ms/step - loss: 0.0208 - accuracy: 0.0025 - val_loss: 0.0315 - val_accuracy: 0.0000e+00
Epoch 6/500
7/7 [==============================] - 0s 27ms/step - loss: 0.0201 - accuracy: 0.0025 - val_loss: 0.0280 - val_accuracy: 0.0000e+00
Epoch 7/500
7/7 [==============================] - 0s 24ms/step - loss: 0.0193 - accuracy: 0.0025 - val_loss: 0.0308 - val_accuracy: 0.

In [3]:
# LSTM_B

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_B
    model_B = Sequential()
    model_B.add(LSTM(32, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_B.add(LSTM(16, return_sequences=True))
    model_B.add(LSTM(8, return_sequences=True))
    model_B.add(Dropout(rate=0.5))
    model_B.add(Flatten())
    model_B.add(Dense(512, activation="relu"))
    model_B.add(Dropout(rate=0.5))
    model_B.add(Dense(64, activation="relu"))
    model_B.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_B.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_B = model_B.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_19 = common_data[common_data['year'] == 2019]

    # 2023년 데이터 스케일링
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_B = model_B.predict(X_2019_seq)

    # 스케일 복원
    y_pred_B = scaler_y.inverse_transform(y_pred_scaled_B)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_19[target].values

    # RMSE 계산
    rmse_B = np.sqrt(mean_squared_error(y_test_actual, y_pred_B))
    rmse_list.append(rmse_B)

    # MAE 계산
    mae_B = mean_absolute_error(y_test_actual, y_pred_B)
    mae_list.append(mae_B)
    
    # MAPE 계산
    mape_B = mean_absolute_percentage_error(y_test_actual, y_pred_B)
    mape_list.append(mape_B)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_B}, MAE: {mae_B}, MAPE: {mape_B}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
7/7 [==============================] - 14s 418ms/step - loss: 0.1167 - accuracy: 0.0025 - val_loss: 0.0320 - val_accuracy: 0.0000e+00
Epoch 2/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0397 - accuracy: 0.0025 - val_loss: 0.0322 - val_accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0272 - accuracy: 0.0025 - val_loss: 0.0473 - val_accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0238 - accuracy: 0.0025 - val_loss: 0.0297 - val_accuracy: 0.0000e+00
Epoch 5/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0234 - accuracy: 0.0025 - val_loss: 0.0303 - val_accuracy: 0.0000e+00
Epoch 6/500
7/7 [==============================] - 0s 17ms/step - loss: 0.0221 - accuracy: 0.0025 - val_loss: 0.0355 - val_accuracy: 0.0000e+00
Epoch 7/500
7/7 [==============================] - 0s 19ms/step - loss: 0.0210 - accuracy: 0.0025 - val_loss: 0.0289 - val_accuracy: 0

In [4]:
# LSTM_C

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_C
    model_C = Sequential()
    model_C.add(LSTM(32, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_C.add(LSTM(32, return_sequences=True))
    model_C.add(BatchNormalization())
    model_C.add(Flatten())
    model_C.add(Dense(512, activation="relu"))
    model_C.add(BatchNormalization())
    model_C.add(Dense(64, activation="relu"))
    model_C.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_C.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_C = model_C.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_19 = common_data[common_data['year'] == 2019]

    # 2023년 데이터 스케일링
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_C = model_C.predict(X_2019_seq)

    # 스케일 복원
    y_pred_C = scaler_y.inverse_transform(y_pred_scaled_C)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_19[target].values

    # RMSE 계산
    rmse_C = np.sqrt(mean_squared_error(y_test_actual, y_pred_C))
    rmse_list.append(rmse_C)

    # MAE 계산
    mae_C = mean_absolute_error(y_test_actual, y_pred_C)
    mae_list.append(mae_C)
    
    # MAPE 계산
    mape_C = mean_absolute_percentage_error(y_test_actual, y_pred_C)
    mape_list.append(mape_C)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_C}, MAE: {mae_C}, MAPE: {mape_C}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
7/7 [==============================] - 12s 324ms/step - loss: 0.2365 - accuracy: 0.0025 - val_loss: 0.2274 - val_accuracy: 0.0000e+00
Epoch 2/500
7/7 [==============================] - 0s 17ms/step - loss: 0.2116 - accuracy: 0.0025 - val_loss: 0.2274 - val_accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 16ms/step - loss: 0.1932 - accuracy: 0.0025 - val_loss: 0.2274 - val_accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 15ms/step - loss: 0.1938 - accuracy: 0.0025 - val_loss: 0.2274 - val_accuracy: 0.0000e+00
Epoch 5/500
7/7 [==============================] - 0s 16ms/step - loss: 0.1918 - accuracy: 0.0025 - val_loss: 0.2274 - val_accuracy: 0.0000e+00
Epoch 6/500
7/7 [==============================] - 0s 16ms/step - loss: 0.1960 - accuracy: 0.0025 - val_loss: 0.2274 - val_accuracy: 0.0000e+00
Epoch 7/500
7/7 [==============================] - 0s 16ms/step - loss: 0.1951 - accuracy: 0.0025 - val_loss: 0.2274 - val_accuracy: 0

In [5]:
# LSTM_D

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_D
    model_D = Sequential()
    model_D.add(Masking(mask_value=0.0, input_shape=(seq_length, X_train.shape[2])))
    model_D.add(LSTM(64, return_sequences=True))
    model_D.add(LSTM(64, return_sequences=True))
    model_D.add(Dropout(rate=0.5))
    model_D.add(TimeDistributed(Dense(10)))
    model_D.add(Flatten())
    model_D.add(Dense(512, activation="relu"))
    model_D.add(Dropout(rate=0.5))
    model_D.add(Dense(64, activation="relu"))
    model_D.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_D.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_D = model_D.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_19 = common_data[common_data['year'] == 2019]

    # 2023년 데이터 스케일링
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_D = model_D.predict(X_2019_seq)

    # 스케일 복원
    y_pred_D = scaler_y.inverse_transform(y_pred_scaled_D)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_19[target].values

    # RMSE 계산
    rmse_D = np.sqrt(mean_squared_error(y_test_actual, y_pred_D))
    rmse_list.append(rmse_D)

    # MAE 계산
    mae_D = mean_absolute_error(y_test_actual, y_pred_D)
    mae_list.append(mae_D)
    
    # MAPE 계산
    mape_D = mean_absolute_percentage_error(y_test_actual, y_pred_D)
    mape_list.append(mape_D)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_D}, MAE: {mae_D}, MAPE: {mape_D}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
7/7 [==============================] - 29s 981ms/step - loss: 0.0926 - accuracy: 0.0025 - val_loss: 0.0382 - val_accuracy: 0.0099
Epoch 2/500
7/7 [==============================] - 0s 24ms/step - loss: 0.0341 - accuracy: 0.0025 - val_loss: 0.0604 - val_accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 23ms/step - loss: 0.0279 - accuracy: 0.0025 - val_loss: 0.0289 - val_accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 23ms/step - loss: 0.0237 - accuracy: 0.0000e+00 - val_loss: 0.0424 - val_accuracy: 0.0000e+00
Epoch 5/500
7/7 [==============================] - 0s 22ms/step - loss: 0.0238 - accuracy: 0.0025 - val_loss: 0.0293 - val_accuracy: 0.0000e+00
Epoch 6/500
7/7 [==============================] - 0s 22ms/step - loss: 0.0222 - accuracy: 0.0025 - val_loss: 0.0342 - val_accuracy: 0.0000e+00
Epoch 7/500
7/7 [==============================] - 0s 23ms/step - loss: 0.0228 - accuracy: 0.0025 - val_loss: 0.0314 - val_accuracy: 0

In [6]:
# LSTM_E

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_E
    model_E = Sequential()
    model_E.add(LSTM(32, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_E.add(LSTM(32, return_sequences=True))
    model_E.add(Dropout(rate=0.5))
    model_E.add(Flatten())
    model_E.add(Dense(512, activation="relu"))
    model_E.add(Dropout(rate=0.5))
    model_E.add(Dense(64, activation="relu"))
    model_E.add(Dense(1, activation='relu'))

    # 컴파일
    adam = optimizers.Adam(learning_rate=0.001)
    model_E.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # 조기 종료 콜백
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # 모델 학습
    history_E = model_E.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # 2023년 데이터 필터링
    data_19 = common_data[common_data['year'] == 2019]

    # 2023년 데이터 스케일링
    X_2019_scaled = scaler_X.transform(data_19[features].values)

    # 시계열 데이터 형태로 변환 (2022년, 2021년, 2020년 데이터를 사용하여 2023년 예측)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2019_seq = create_sequences_for_prediction(X_2019_scaled, seq_length)

    # 2023년 데이터 예측
    y_pred_scaled_E = model_E.predict(X_2019_seq)

    # 스케일 복원
    y_pred_E = scaler_y.inverse_transform(y_pred_scaled_E)

    # 실제 값과 예측 값 비교를 위해 실제 2023년 p_era 값도 복원
    y_test_actual = data_19[target].values

    # RMSE 계산
    rmse_E = np.sqrt(mean_squared_error(y_test_actual, y_pred_E))
    rmse_list.append(rmse_E)

    # MAE 계산
    mae_E = mean_absolute_error(y_test_actual, y_pred_E)
    mae_list.append(mae_E)
    
    # MAPE 계산
    mape_E = mean_absolute_percentage_error(y_test_actual, y_pred_E)
    mape_list.append(mape_E)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_E}, MAE: {mae_E}, MAPE: {mape_E}')

# 평균 RMSE, MAE, MAPE 계산
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
7/7 [==============================] - 13s 366ms/step - loss: 0.0946 - accuracy: 0.0025 - val_loss: 0.0394 - val_accuracy: 0.0099
Epoch 2/500
7/7 [==============================] - 0s 21ms/step - loss: 0.0369 - accuracy: 0.0025 - val_loss: 0.0621 - val_accuracy: 0.0000e+00
Epoch 3/500
7/7 [==============================] - 0s 19ms/step - loss: 0.0354 - accuracy: 0.0025 - val_loss: 0.0388 - val_accuracy: 0.0000e+00
Epoch 4/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0263 - accuracy: 0.0025 - val_loss: 0.0297 - val_accuracy: 0.0000e+00
Epoch 5/500
7/7 [==============================] - 0s 21ms/step - loss: 0.0224 - accuracy: 0.0025 - val_loss: 0.0411 - val_accuracy: 0.0000e+00
Epoch 6/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0230 - accuracy: 0.0025 - val_loss: 0.0351 - val_accuracy: 0.0000e+00
Epoch 7/500
7/7 [==============================] - 0s 16ms/step - loss: 0.0230 - accuracy: 0.0025 - val_loss: 0.0330 - val_accuracy: 0.000

In [7]:
y_pred_A_list = [item[0] for item in y_pred_A]

# 출력 옵션 설정
pd.set_option('display.max_rows', None)

df = pd.DataFrame({'Actual': data_19[target], 'Predicted': y_pred_A_list})
df['diff'] = df['Predicted'] - df['Actual']
print(df.sort_values(by='Actual'))

      Actual  Predicted      diff
1765    1.19   2.046806  0.856806
1770    1.50   2.114560  0.614560
1857    1.65   2.520537  0.870537
1869    1.87   2.328091  0.458091
1775    1.91   2.640804  0.730804
1854    2.21   2.076632 -0.133368
1883    2.30   2.829885  0.529885
1803    2.32   2.397560  0.077560
1931    2.43   2.925905  0.495905
1716    2.45   3.258264  0.808264
1834    2.47   3.015399  0.545399
1862    2.48   2.661267  0.181267
1824    2.50   1.837969 -0.662031
1741    2.54   3.467897  0.927897
1707    2.58   2.602662  0.022662
1886    2.61   2.688837  0.078837
1815    2.63   2.525716 -0.104284
1711    2.68   3.493928  0.813928
1998    2.70   2.172056 -0.527944
1962    2.71   2.301654 -0.408346
1706    2.71   2.395446 -0.314554
1787    2.80   2.739596 -0.060404
1830    2.87   2.707650 -0.162350
1745    2.90   2.512711 -0.387289
1729    2.92   2.887081 -0.032919
1851    2.92   2.979100  0.059100
1925    2.93   2.697302 -0.232698
1702    2.93   2.752888 -0.177112
1946    2.94  